<a href="https://colab.research.google.com/github/Dimka27/Projects/blob/main/%D0%9F%D1%80%D0%BE%D0%BF%D0%B0%D0%B2%D1%88%D0%B8%D0%B9_%D1%81%D0%B0%D0%BC%D0%BE%D0%BB%D0%B5%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!unzip '/content/drive/MyDrive/Пропавшие самолеты/avia-train.zip'
!unzip '/content/drive/MyDrive/Пропавшие самолеты/avia-test.zip'

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from google.colab.patches import cv2_imshow
from sklearn.metrics import roc_auc_score
from google.colab import files

# Download

In [ ]:
DIR_csv = '/content/drive/MyDrive/Пропавшие самолеты/'
DIR_img = '/content/'

In [ ]:
train_df = pd.read_csv(DIR_csv + 'train.csv')
test_df = pd.read_csv(DIR_csv + 'test.csv')

In [ ]:
train_df.head()

,sign,filename
0,0,200d8766-4240-44d1-a2da-65dc4ee7973a
1,0,87174854-b7b6-4cb2-8183-4ca2bbd26b9d
2,0,c089638e-b683-436a-a3a3-3e5be972c4f6
3,1,8790e76c-f485-49a6-abaf-3147436ca085
4,0,c06b01db-8934-4c72-b568-508a32c4b170


In [ ]:
test_df.head()

,filename
0,68f33844-472b-4111-b600-f90d544833c7
1,7d93a21d-1f16-49ce-8fcc-edf12c40f549
2,4a820650-7acd-489a-ad14-9d7ad8c73b6b
3,819b216b-2b6c-4539-a722-70648c0706c6
4,45f7c47d-03cc-40cd-acc5-b8c1c57872fa


In [ ]:
train_df.values

array([[0, '200d8766-4240-44d1-a2da-65dc4ee7973a'],
       [0, '87174854-b7b6-4cb2-8183-4ca2bbd26b9d'],
       [0, 'c089638e-b683-436a-a3a3-3e5be972c4f6'],
       ...,
       [1, '4ac0df6f-64cb-4b7a-8325-7b9bdfb587ef'],
       [1, '8eef88ec-3680-49ce-876f-86f52d14f162'],
       [0, 'db2543e0-ca73-4c7f-9203-2f8f3f6b3158']], dtype=object)

In [ ]:
train_df[train_df['sign'] == 1]

,sign,filename
3,1,8790e76c-f485-49a6-abaf-3147436ca085
5,1,56a03a71-b4c6-406c-b987-b4c55c0dff8c
30,1,c5ff626f-e486-4245-a883-b00c9b672ce0
34,1,b7afbfbf-0b76-4e61-8aad-b2474821331e
45,1,303f36d3-8d7d-48e9-acc3-1f2634052581
...,...,...
31066,1,75e51a54-8356-49c2-8272-605cb1e4329a
31068,1,61de2074-eb9a-4c87-b45c-2be7a912444c
31075,1,5fc20b8d-8773-40ee-8d1b-6fe27f10aa2a
31077,1,4ac0df6f-64cb-4b7a-8325-7b9bdfb587ef


In [ ]:
p1 = '/content/avia-train/0f2a986d-9ff8-4b8e-8a50-d5bcafb39c5a.png'
p2 = '/content/avia-train/01e425cb-f365-4250-9a03-961c37fe76f1.png'

In [ ]:
img1 = cv2.imread(p1)
img2 = cv2.imread(p2)

# Dataset

In [ ]:
%%capture
!pip install opencv-torchvision-transforms-yuzhiyang
from cvtorchvision import cvtransforms

In [ ]:
class SquarePad:
    def __call__(self, image):
        max_wh = max(image.shape)
        p_left, p_top = [(max_wh - s) // 2 for s in image.shape[:2]]
        p_right, p_bottom = [max_wh - (s+pad) for s, pad in zip(image.shape[:2], [p_left, p_top])]
        padding = (p_left, p_top, p_right, p_bottom)
        trans = cvtransforms.Pad(padding, fill=(0, 0, 0))
        return trans(image)

In [ ]:
IMAGE_SIZE = (299, 299) #(40, 40)

class CustDataset(Dataset):
    def __init__(self, pathes, mode):
        self.pathes = pathes
        self.mode = mode
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]

    def __len__(self):
        return len(self.pathes)

    def __getitem__(self, ind):
        path = DIR_img + 'avia-' + self.mode + '/' + self.pathes[ind][-1] + '.png'
        label = self.pathes[ind][0] if self.mode == 'train' else None

        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #img = cv2.resize(img, IMAGE_SIZE)

        transform = self.transform()

        output = \
            {
                "train": 
                    {
                        "label": label,
                        "image": transform(img),
                        "path": path
                    },
             
                "test": 
                    {
                        "image": transform(img),
                        "path": path
                    }
            }

        return output[self.mode]

    def transform(self):
        transform = \
            {
                "train": cvtransforms.Compose
                    ([
                        #cvtransforms.Resize(IMAGE_SIZE),
                        #cvtransforms.RandomCrop(32, padding=4),
                        #self.pad_image(),
                        cvtransforms.Resize((100, 100)),
                        SquarePad(),
                        cvtransforms.Resize(IMAGE_SIZE),
                        cvtransforms.RandomHorizontalFlip(),
                        cvtransforms.RandomVerticalFlip(),
                        #cvtransforms.RandomRotation(degrees=(0, 180)),
                        #cvtransforms.RandomAffine(degrees=(30, 70), translate=(0.1, 0.3), scale=(0.5, 0.75)),
                        cvtransforms.ToTensor(),
                        cvtransforms.Normalize(mean=self.mean, std=self.std),
                    ]),
             
                "test": cvtransforms.Compose
                    ([
                        #cvtransforms.Resize(IMAGE_SIZE),
                        #self.pad_image(),
                        cvtransforms.Resize((100, 100)),
                        SquarePad(),
                        cvtransforms.Resize(IMAGE_SIZE),
                        cvtransforms.ToTensor(),
                        cvtransforms.Normalize(mean=self.mean, std=self.std),
                    ]),
            }

        return transform[self.mode]

In [ ]:
train_set, val_set = train_test_split(train_df.values, test_size=0.15, shuffle=True, random_state=42)
train_set = CustDataset(train_set, 'train')
val_set = CustDataset(val_set, 'train')
test_set = CustDataset(test_df.values, 'test')

# Loaders

In [ ]:
BATCH_SIZE = 32
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

# inception_resnet_v2

In [ ]:
%%capture
!pip install pretrainedmodels
import pretrainedmodels

In [ ]:
model_name = 'inceptionresnetv2' # fbresnet152 | inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/inceptionresnetv2-520b38e4.pth" to /root/.cache/torch/hub/checkpoints/inceptionresnetv2-520b38e4.pth


In [ ]:
model.add_module(
    'final_block',
    nn.Sequential(
        nn.Linear(1000, 2),
    )
)

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

In [ ]:
model.final_block.apply(init_weights)

Sequential(
  (0): Linear(in_features=1000, out_features=64, bias=True)
  (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=64, out_features=256, bias=True)
  (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
named_parameters = []
for par in model.named_parameters():
    named_parameters.append(par[0])

In [ ]:
named_parameters

In [ ]:
freeze = named_parameters.index('repeat_1.19.branch0.conv.weight')
freeze

514

In [ ]:
cnt_parameters = 0
for par in model.parameters():
    cnt_parameters = cnt_parameters + 1
    if cnt_parameters < freeze:
        par.requires_grad = False
    else:
        break

cnt_parameters

514

# Train

In [ ]:
MIN_LOSS = 0.5
PATH_MODELS = 'inceptionresnetv2_best.pth'

In [ ]:
def train(model, train_loader, val_loader, n_epoch, criterion, optimizer, scheduler, MIN_LOSS):
    model.cuda()
    model.train()
    for epoch in range(n_epoch):
        epoch_loss = 0
        roc_auc = 0
        items = 0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
        pbar.set_description(f"Epoch {epoch + 1}")
        for ind, batch in pbar:
            with torch.set_grad_enabled(True):

                img = batch['image'].cuda()
                label = batch['label']
                #print(img.shape)

                out = model(img).cpu()
                optimizer.zero_grad()

                loss = criterion(out, label)

                loss.backward()
                
                optimizer.step()

                epoch_loss += loss.item()
                try:
                    roc_auc += roc_auc_score(np.array(label), torch.argmax(out, axis=1).numpy()) #sum(torch.argmax(out, axis=1).numpy() == np.array(label))
                    #roc_auc += roc_auc_score(np.array(label), np.array(list(map(lambda x: 1 if x > 0.6 else 0, out)))) #sum(torch.argmax(out, axis=1).numpy() == np.array(label))
                except:
                    roc_auc = -1
                items += 1

        roc_auc /= items
        epoch_loss /= len(train_loader)
        val, val_loss, roc_auc_val = valid(model, val_loader, n_epoch, criterion)

        if val_loss < MIN_LOSS:
            torch.save(model.state_dict(), PATH_MODELS)
            MIN_LOSS = val_loss

        print(f'Epoch: {epoch + 1} Train Loss: {epoch_loss} ROC AUC: {roc_auc} {val}')
        
        scheduler.step()
    return model, MIN_LOSS, (epoch_loss, val_loss, roc_auc, roc_auc_val)

In [ ]:
def valid(model, loader, n_epoch, criterion):
    model.cuda()
    model.eval()
    total_loss = 0
    roc_auc = 0
    items = 0
    pbar = tqdm(enumerate(loader), total=len(loader), leave=False)
    for ind, batch in pbar:
        with torch.no_grad():

            img = batch['image'].cuda()
            label = batch['label']

            out = model(img).cpu()
            optimizer.zero_grad()

            loss = criterion(out, label)

            total_loss += loss.item()

            out = torch.argmax(out, axis=1).numpy()
            try:
                roc_auc += roc_auc_score(np.array(label), out)
                #roc_auc += roc_auc_score(np.array(label), np.array(list(map(lambda x: 1 if x > 0.6 else 0, out))))
            except:
                roc_auc = -1
            items += 1


    roc_auc /= items
    total_loss /= len(loader)
    #accuracy = "{:.2f}".format(accuracy * 100)
    return f'Loss: {total_loss} ROC AUC: {roc_auc}', total_loss, roc_auc

In [ ]:
def test(model, loader):
    model.cuda()
    model.eval()
    pbar = tqdm(enumerate(loader), total=len(loader), leave=False)
    prediction = []
    pathes = []
    for ind, batch in pbar:
        with torch.no_grad():

            img = batch['image'].cuda()
            path = batch['path']

            out = model(img).cpu()

            #out = np.array(list(map(lambda x: 1 if x > 0.6 else 0, out)))
            out = torch.argmax(out, axis=1).tolist()
            #print(out.shape, out)
            prediction += out
            pathes += path

    return prediction, pathes

# Training

In [ ]:
n_epoch = 12
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=3e-3)
Q = np.floor(len(train_set)/BATCH_SIZE)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, Q)

torch.cuda.empty_cache()

model = train(model, train_loader, val_loader, n_epoch, criterion, optimizer, scheduler)

TypeError: ignored

# Training cross-validation

In [ ]:
skf = StratifiedKFold(n_splits=5,  random_state=42)
skf.get_n_splits(train_df['filename'].values, train_df['sign'].values)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


5

In [ ]:
full_train_set = CustDataset(train_df.values, 'train')

In [ ]:
def colate_fn(batch):
    new_batch = {'image': [], 'label': [], 'path': []}
    for ind in batch:
        x = full_train_set[ind]
        new_batch['image'].append(x['image'])
        new_batch['label'].append(x['label'])
        new_batch['path'].append(x['path'])

    new_batch['image'] = torch.stack(new_batch['image'])
    new_batch['label'] = torch.LongTensor(new_batch['label'])
    #new_batch['path'] = torch.Tensor(new_batch['path'])
    return new_batch

In [ ]:
torch.cuda.empty_cache()
MIN_LOSS = 1
TOTAL_HISTORY = {'train loss': [], 'val loss': [], 'train ROC AUC': [], 'val ROC AUC': []}

n_epoch = 4
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=3e-3)
Q = np.floor(len(train_set)/BATCH_SIZE)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, Q)

for train_index, val_index in skf.split(train_df['filename'].values, train_df['sign'].values):
    print("TRAIN:", train_index, "TEST:", val_index)

    train_loader = DataLoader(train_index, batch_size=BATCH_SIZE, shuffle=True, drop_last=False, collate_fn=colate_fn)
    val_loader = DataLoader(val_index, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, collate_fn=colate_fn)

    model, MIN_LOSS, history_value = train(model, train_loader, val_loader, n_epoch, criterion, optimizer, scheduler, MIN_LOSS)

    TOTAL_HISTORY['train loss'].append(history_value[0])
       TOTAL_HISTORY['val loss'].append(history_value[1])
    TOTAL_HISTORY['train ROC AUC'].append(history_value[2])
    TOTAL_HISTORY['val ROC AUC'].append(history_value[3])



TRAIN: [ 6213  6214  6217 ... 31077 31078 31079] TEST: [   0    1    2 ... 6215 6216 6230]


Epoch: 1 Train Loss: 0.18535193101954833 ROC AUC: 0.9247635544099488 Loss: 0.13616925285747036 ROC AUC: 0.9211278682046466


Epoch: 2 Train Loss: 0.7121746463166547 ROC AUC: 0.5033728272444629 Loss: 0.5791852643856635 ROC AUC: 0.5


Epoch: 3 Train Loss: 1.4899743201338147 ROC AUC: 0.24406009570580955 Loss: 2.5958453834056856 ROC AUC: 0.4941177297859524


Epoch: 4 Train Loss: 11.908271551141791 ROC AUC: 0.6136952243887253 Loss: 0.14537246904980677 ROC AUC: 0.9420830897889173
TRAIN: [    0     1     2 ... 31077 31078 31079] TEST: [ 6213  6214  6217 ... 12461 12462 12466]


...

Epoch: 4 Train Loss: 0.04311469948101514 ROC AUC: 0.9823790969318633 Loss: 0.04125631774047342 ROC AUC: -0.005128205128205128
TRAIN: [    0     1     2 ... 25039 25041 25048] TEST: [24795 24796 24797 ... 31077 31078 31079]


Epoch: 1 Train Loss: 0.05770644615329395 ROC AUC: 0.9758601488752755 Loss: 0.03035004019486503 ROC AUC: 0.9594025983719408


Epoch: 2 Train Loss: 0.049990719404524646 ROC AUC: 0.9758876483086948 Loss: 0.030247909664840568 ROC AUC: 0.9566587250488763


Epoch: 3 Train Loss: 0.04089106456041375 ROC AUC: 0.044090771544187696 Loss: 0.03270069811006602 ROC AUC: 0.9582500348185188


Epoch: 4 Train Loss: 0.03590947405116744 ROC AUC: 0.982874694568322 Loss: 0.031158220872450144 ROC AUC: 0.9598650876286957


# Prediction

In [ ]:
cnt_parameters = 0
for par in model.parameters():
    cnt_parameters =+ 1
    if cnt_parameters < freeze:
        par.requires_grad = True
    else:
        break

In [ ]:
for par in model.parameters():
    par.requires_grad = True

In [ ]:
model.load_state_dict(torch.load(PATH_MODELS))

<All keys matched successfully>

In [ ]:
prediction, pathes = test(model, test_loader)

In [ ]:
prediction

In [ ]:
pathes = list(map(lambda s: s.split('/')[-1].split('.')[0], pathes))

In [ ]:
pred_df = pd.DataFrame(data={'sign': prediction, 'filename': pathes})
pred_df.set_index('sign', inplace=True)

In [ ]:
pred_df.to_csv('submission.csv')

In [ ]:
from google.colab import files

files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download(PATH_MODELS)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>